In [1]:
# Import liberaries and functions
import pandas as pd
import numpy as np

In [2]:
rf_rates = pd.read_csv("Data\\rf_rates.csv")

rf_rates["date"] = pd.to_datetime(rf_rates["date"])
rf_rates.set_index("date", inplace=True)

mktrf > Excess Return on the Market\
rf > Risk-Free Return Rate (One Month Treasury Bill Rate)

In [3]:
prices_df = pd.read_csv("Data\Prices.csv")
prices_df["Date"] = pd.to_datetime(prices_df["Date"])
prices_df.set_index('Date', inplace=True)
prices_df.columns

Index(['Instrument', 'CLOSEPRICE', 'VOLUME', 'COMPANYMARKETCAP',
       'TTLCMNSHARESOUT'],
      dtype='object')

In [4]:
# Calculate returns
prices_df["Returns"] = prices_df.groupby("Instrument")["CLOSEPRICE"].pct_change(1)

In [5]:
Beta_df = pd.merge(
    left=prices_df,
    right=rf_rates,
    left_on='Date',
    right_index=True,
    how='left'
)

Beta_df["ex_Returns"] = Beta_df["Returns"] - Beta_df["rf"]

In [6]:
N = 30

Beta_df[f"mktrf_var_{N}"] = (
    Beta_df.groupby('Instrument')["mktrf"]
    .rolling(N, min_periods=N//2).var()
).values

Beta_df[f"covar_{N}"] = (
    Beta_df.groupby("Instrument")[["mktrf", "ex_Returns"]]
    .rolling(N, min_periods=N//2).cov()
    .unstack()["mktrf"]["ex_Returns"]
).values

Beta_df[f"Beta_{N}"] = Beta_df[f"covar_{N}"] / Beta_df[f"mktrf_var_{N}"]

In [7]:
N = 90

Beta_df[f"mktrf_var_{N}"] = (
    Beta_df.groupby('Instrument')["mktrf"]
    .rolling(N, min_periods=N//2).var()
).values

Beta_df[f"covar_{N}"] = (
    Beta_df.groupby("Instrument")[["mktrf", "ex_Returns"]]
    .rolling(N, min_periods=N//2).cov()
    .unstack()["mktrf"]["ex_Returns"]
).values

Beta_df[f"Beta_{N}"] = Beta_df[f"covar_{N}"] / Beta_df[f"mktrf_var_{N}"]

In [8]:
# Abnormal Return = Actual Return - Expected Return 
# Estimation window: N days before the event
Beta_df["AR"] = Beta_df["Returns"] - (
    Beta_df["rf"] + Beta_df.groupby("Instrument")["Beta_90"].shift(15)*Beta_df["mktrf"]
)

In [9]:
# Cumulative AR - Event window: -2 and +5 days 
Beta_df["CAR"] = (
    Beta_df.groupby("Instrument")["AR"].rolling(8, min_periods=5).mean()
    .groupby("Instrument").shift(-5)
).values

In [14]:
Beta_df.reset_index(inplace=True)

Beta_df.drop(
    columns=[
        'CLOSEPRICE', 'VOLUME', 'COMPANYMARKETCAP', 'TTLCMNSHARESOUT', 
        'Returns', 'mktrf', 'rf', 'mktrf_var_30', 'covar_30', 'mktrf_var_90', 'covar_90'
    ],
    inplace=True
)

In [21]:
Beta_df.isna().sum()

Date                0
Instrument          0
ex_Returns    1845903
Beta_30        926407
Beta_90       1141443
AR            2207961
CAR           1336689
dtype: int64

In [22]:
Beta_df.to_csv("Data\Beta_AR.csv", index=False)

In [2]:
market_idx = pd.read_csv('Data\market_daily.csv')

market_idx["DATE"] = pd.to_datetime(market_idx["DATE"])
market_idx.set_index("DATE", inplace=True)

vwretd > Value-Weighted Return (includes distributions)\
vwretx > Value-Weighted Return (excluding dividends)\
ewretd > Equal-Weighted Return (includes distributions)\
ewretx > Equal-Weighted Return (excluding dividends)\
totval > Total Market Value\
totcnt > Total Market Count\
usdval > Market Value of Securities Used\
usdcnt > Count of Securities Used\
sprtrn > Return on S&P Composite Index\
spindx > Level on S&P Composite Index